<a href="https://colab.research.google.com/github/a-forty-two/DataSetsForML/blob/master/18_Activity_Detection_and_TRANSFER_LEARNING.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# we will use a hardcoded video as test data
# if doing on laptop you can even use webcab for same program 
# as input stream 

# inputs were hardcoded,video is not a user input but an mp4 or avi file

# Algorithm
# 1. Input a video (or live stream)
# 2. open the frames as images
# 3. Run object detection on images to understand continous usage of objects (RESNET/VGG)
# 4. Continous usage of objects is called ACTIVITY
# 5. in each frame of video, classify the frame as an activity
# 6. combine the frames back into a video to deliver as classified output! 



In [0]:
######## start of: TRAIN.py


from keras.preprocessing.image import ImageDataGenerator 
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import pickle # to read/write binary files as dumps (for weights and biases or binarized labels)
import cv2
import os # os.separators to separate path names and get file name and file folder name
from keras.applications import ResNet50
# to customize models with own layers
from keras.layers.core import Flatten, Dropout, Dense
from keras.layers.pooling import AveragePooling2D # AVERAGE pixel -> do this when color of object shouldn't matter 
from keras.layers import Input # input layer can take general or any input size (dynamic input size but once decalred cannot be changed)
from keras.models import Model
#label binarizer to encode/decode binarized outputs 
from sklearn.preprocessing import LabelBinarizer
import imutils
from imutils import paths
# but what if i wanted to take average without losing channels?
# Conv2D -> GlobalAveragePooling-> Conv2D would split data into R, G and B channels 
# then GAP will calculate averages for R, G, and B separately thus preserving colors 
from keras.optimizers import SGD
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split


In [0]:
# variables that should have been collected from user through UI or command line or API post
dataset = 'data'
model_path = 'bin'
binarizer_path = 'bin'
evaluation_path = 'eval'
test_path = 'test'

In [23]:
# shuffle the data after reading all paths
import random
imagePaths = sorted(list( paths.list_images(dataset)))
random.seed(42)
random.shuffle(imagePaths)
imagePaths[:2]

['data/hockey/00000065.jpg', 'data/weightlifting/00000072.jpg']

In [0]:
# need to conver to img_to_array and resize 
# this is how to use ResNET 
# in our VGG-> we took (96,96,3)
# larger images are required because smaller image= not enough filters possible
label_names = ["weightlifting","swimming","hockey","basketball"]
labels = []
data = []
for path in imagePaths:
  label = path.split(os.path.sep)[-2] # folder name! to represent class name 
  if label not in label_names:
    continue
  labels.append(label)
  img = cv2.imread(path)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)   # these are simple reshape, resize and expand_dims functions
  img = cv2.resize(img, (224,224)) 
  data.append(img)
  # RESNET model -> input size (224,224,3) , only accept RGB channel, while most images
  # like jpeg, png, jpg, giff are arranged in BGR (just opposite)
  # Object detection -> there order of the color did  not matter because RGB values combined would have 
  # resulted in same color anyway 
  # just in case error handling-> if image is already RGB, the below function will do nothing, if its BGR, it will
  # be converted to RGB 
  
  # they readjust dimensions to fit various frameworks 
  # RESNET doesn't require image aug sep because it was inbuilt into its architecture 
  # aug.flow-> model.fit_generator 
  

In [0]:

data = np.array(data)
labels = np.array(labels)
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels # HOT-encoded MATRIX 
(xtrain, xtest, ytrain, ytest) = train_test_split(data, labels, test_size=0.2, stratify=labels, random_state=42 )
# << left shift operation
#       0 0 0 1  0 0 0 0   -> left shift by 2 - >   0 1 0 0 0 0 0 0 
#          -> right shift by 1 -> 0000 1000 
# This label binarizing is EXTREMELY FAST because of LEFT shift operations which happen at bit-level 
# there is no need to go through the entire list of items, just need to know HOW many items

In [0]:
# split into train/test
# whenever we need classes balanced while splitting, we use a process called 'stratification'
# it shuffles on the basis of types of output labels, and not just random_state
(xtrain, xtest, ytrain, ytest) = train_test_split(data, labels, test_size=0.2, stratify=labels, random_state=42 )

In [0]:
trainAug = ImageDataGenerator(rotation_range=25, zoom_range=0.20, width_shift_range=0.2, height_shift_range=0.2, 
                              shear_range=0.20, horizontal_flip=True, fill_mode='nearest')

valAug = ImageDataGenerator() # default augmentation (everything is set to false-> no augmentation)
# this will help validate the model in 2 augmentation- one with our v/s with any other option! 

means = np.array( [ 103.939, 116.779, 123.68 ])
trainAug.mean = means
valAug.mean = means 
# deviations will be calculated from respective means
# train data -> trainaug.means
# valdata -> valAug.means 

# Since the data is augmented and no longer the original image, the difference between the original image (no aug- val)
# and augmented image (trainAug) can be calculated as difference in the mean 
# when using keras.application 
# To find out this mean there are various formula, but we are going to use the preferred production method-
# WITH RESNET and VGG, we use means from 'ImageNet'. 
# 3 averages-> 1 each for R, G and B 
# From scratch way is to calculate it based on your data. BUT, when doing transfer learning, follow what model
# says. VGG and Restnet for 'ImageNet' standards. 




In [30]:
# LAST time-> we built our own PandaVGG, THIS time-> transfer learning using ResNet 
# Last time-> we calculated our weights for PandaVGG, THIS TIME-> preserved ResNet's weights by setting 
# non trainable layers 
# REST of it is EXACTLY the same except hyperparameters 






# cloud -> hadoop (parquet) -> g(s3) -> no bigquery
# PySpark notebooks -> external tables -> DataBricks 
# Transfer learning and build our model on top of it
basemodel = ResNet50(weights='imagenet', include_top=False, input_tensor=Input(shape=(224,224,3)))
# activity detection and classification model
# use object detection as base algorithm and build your own algo on top of it 
# basemodel was loaded as KNOWLEDGE
myModel = basemodel.output # TRANSFER LEARNING (Knowledge transfer) is done 
# another of adding layers besides model.add
# MATMUL way -> multiplicative way of adding layers (because anyway between layers its the mul, inside layers-> add)
# model  = (additive_layers)(model)
myModel = AveragePooling2D(pool_size=(7,7))(myModel) # in matmul AXB != BXA
#Pattern recognition 
# till now my data is 2 d! Image Pattern Recog
# Flatten -> Dense(ReLu) -> Dense (Softmax)
myModel = Flatten(name='flatten')(myModel) # specific names to layers, you can do so by name property
myModel = Dense(512, activation='relu')(myModel)
myModel = Dense(len(lb.classes_), activation='softmax')(myModel)

model = Model(inputs=basemodel.input, outputs=myModel) # connection of the two pipes -> ResNet and Mymodel 

for layer in basemodel.layers:
  layer.trainable = False 
# Learning will be ONLY for myModel not for basemodel-> resnet weights will be preserved, while myModel will keep 
# learning 
# PARTIAL LEARNING-> some part of model is already smart, some part is learning from it's smartness 



# IF NOW YOU WANT MODEL TO BE FAMILIAR WITH YOUR IMAGES, you need to show images again!
# but this brings us a problem- if i retain the model -> then the imagenet weights will be recalculated!

# every time back-prop -> weights and bias are recalculated
# everything that i inherited from resnet will GO AWAY except its shape!!
# TO PRESERVE THE WEIGHTS INHERITED you can set that layers will NOT be learnt upon
# thAT means their parameters will be unaffected by gradient descent 









/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '
